In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

import torch
import torch.nn as nn

from data.mvtec import *
from meanshift import *


MVTEC_PATH="../../../../nas-files/mvtec"


In [2]:
from sklearn.svm import OneClassSVM

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = models.efficientnet_b4(pretrained=True).features
net = net.to(device)
net = net.eval()

def flatten(X):
    return X.reshape(len(X), -1)

In [4]:
log = []
layers = [5, 6]

for n in [1, 5, 10, 25, 50, 80, 100]:

    for run in range(2):

        for clazz in range(0, 15):

            X_, X_valid_, X_test_, X_labels_, T = dataloader(clazz, P=224, s=224, label_per_patch=False, load_size=256, crop_size=224, normalize=False, MVTEC_PATH=MVTEC_PATH, fold=run )
            X_ = X_[np.random.permutation(len(X_))][:n]
            
            X = concat_features(X_, net, blocks=layers, fmap_pool=False)
            X_valid = concat_features(X_valid_, net, blocks=layers, fmap_pool=False)
            X_test = concat_features(X_test_, net, blocks=layers, fmap_pool=False)

            clf = OneClassSVM(gamma='auto').fit(flatten(X))
            s_in = -clf.score_samples(flatten(X_valid))
            s_out = -clf.score_samples(flatten(X_test))

            auc = roc_auc_score([0] * len(X_valid) + [1] * len(X_test), np.concatenate([s_in, s_out]))

            log.append(pd.DataFrame(np.asarray([run, clazz, layers[0], n, auc])[:, None].T, columns=["run", "class", "layers", "n_samples", "auc"])) 
    

In [5]:
dat = pd.concat(log)

x = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby("n_samples").mean().index
y = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby("n_samples").mean()["auc"]
s = dat[["run", "class", "auc", "n_samples"]].astype(np.float32).groupby(["run", "n_samples"]).mean().groupby([ "n_samples"]).std()["auc"]

np.asarray(x), np.asarray(y), np.asarray(s)

(array([  1.,   5.,  10.,  25.,  50.,  80., 100.]),
 array([0.5781456 , 0.62865186, 0.6516684 , 0.6665207 , 0.6735555 ,
        0.6905192 , 0.69877046], dtype=float32),
 array([0.00816637, 0.00409292, 0.01393459, 0.00899292, 0.00876924,
        0.00249585, 0.00780391]))

In [4]:
# GLOBAL layer 5/6
log_ = pd.concat(log)

for clazz in range(15):
    auc = log_[(log_["class"] == clazz)].groupby("run").mean()["auc"]
    print(f"class={MVTEC.CLASSES[clazz]:10s}", f"\tauc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")
    
log_ = pd.concat(log)

auc = log_.groupby("run").mean()["auc"]
print(f"avg.-auc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")

class=bottle     	auc: 0.992 ± 0.004
class=carpet     	auc: 0.973 ± 0.025
class=leather    	auc: 0.961 ± 0.011
class=pill       	auc: 0.500 ± 0.000
class=tile       	auc: 0.495 ± 0.004
class=wood       	auc: 0.500 ± 0.000
class=cable      	auc: 0.500 ± 0.000
class=grid       	auc: 0.596 ± 0.022
class=toothbrush 	auc: 0.999 ± 0.002
class=zipper     	auc: 0.959 ± 0.013
class=capsule    	auc: 0.842 ± 0.037
class=hazelnut   	auc: 0.512 ± 0.000
class=metal_nut  	auc: 0.615 ± 0.015
class=screw      	auc: 0.758 ± 0.090
class=transistor 	auc: 0.610 ± 0.022
avg.-auc: 0.721 ± 0.005
